# Pràctica 4 PLH - Rubén Álvarez Aragonés i Pol Pérez Prades

____

# Stuff

## Imports 

In [2]:
%pip install -r requirements.txt

  Using cached scipy-1.10.1-cp310-cp310-win_amd64.whl.metadata (58 kB)
  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ml_dtypes-0.3.2-cp310-cp310-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached pydantic-1.10.15-cp310-cp310-win_amd64.whl.metadata (152 kB)
Using cached scipy-1.10.1-cp310-cp310-win_amd64.whl (42.5 MB)
Using cached pydantic-1.10.15-cp310-cp310-win_amd64.whl (2.1 MB)
Using cached smart_open-6.4.0-py3-none-any.whl (57 kB)
Using cached ml_dtypes-0.3.2-cp310-cp310-win_amd64.whl (127 kB)
Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl (413 kB)
  Running setup.py clean for fasttext
Failed to build fasttext
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      C:\Users\Pol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\setuptools\dist.py:476: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              By 2024-Sep-26, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************

In [3]:
import tensorflow as tf
import numpy as np
import nltk
import re
import spacy
from spacy.lang.ca.examples import sentences 
from gensim.models import word2vec
import torch
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Requisites
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')

spacy.cli.download("ca_core_news_sm")


KeyboardInterrupt: 

## Entrenament model Word2Vec

In [ ]:
class Word2VecEmbedder:
    def __init__(self, corpus_path, corpus_size, load_model=False, model_path=None):
        if not load_model:
            self.corpus_path = corpus_path
            self.corpus_size = int(corpus_size * 2**30)  # Convert GB to bytes
            self.corpus = self.get_corpus(corpus_path)
            self.fit()
        else:
            try:
                self.load(model_path)
            except FileNotFoundError:
                print("Model not found. Please check the path.")
                return

    def get_corpus(self, corpus_path):
        with open(corpus_path, 'r', encoding='utf-8') as f:
            print("Reading corpus...")
            corpus = f.read(self.corpus_size)
            print("Preprocessing corpus...")
            corpus = self.preprocess(corpus)  # Preprocess the corpus and tokenize it
        return corpus

    def fit(self, window_size=15, vector_size=300, min_count=10, workers=8, epochs=10):
        # Initialize the Word2Vec model with gensim
        print("Initializing Word2Vec model...")
        self.model = word2vec.Word2Vec(sentences=[self.corpus], vector_size=vector_size, window=window_size, min_count=min_count, workers=workers, epochs=epochs)
        print("Model training completed.")

    def save(self, model_path):
        # Save the model
        self.model.save(model_path)

    def load(self, model_path):
        # Load the model
        self.model = word2vec.Word2Vec.load(model_path)

    def preprocess(self, corpus):
        # Lowercase the corpus
        print("Lowercasing...")
        corpus = corpus.lower()
        
        # Remove special characters
        print("Removing special characters...")
        corpus = re.sub(r'[^a-záàéèíìóòúùñüç\s]', ' ', corpus)
        
        # Tokenize the corpus
        print("Tokenizing...")
        corpus = nltk.word_tokenize(corpus)
        
        # Eliminate last token (probably incomplete word)
        corpus = corpus[:-1]
        
        return corpus

    def get_embedding(self, word):
        # Get the embedding of a word
        try:
            return self.model.wv[word]
        except KeyError:
            print(f"Word '{word}' not in vocabulary.")
            return None

    def print_vocab(self):
        print("Vocabulary:", list(self.model.wv.index_to_key))
        
    

### 1. Model amb 100MB de dades

In [ ]:
Word2Vec_model = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.1)

In [ ]:
Word2Vec_model.save('models/word2vec_model.bin')

In [ ]:
Word2Vec_model.print_vocab()

In [ ]:
Word2Vec_model.get_embedding("de")

In [ ]:
Word2Vec_model.model.wv.most_similar("negre")

### 2. Model amb 500MB de dades

In [ ]:
Word2Vec_model_500 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.5)

In [ ]:
Word2Vec_model.get_embedding("hola")

In [ ]:
Word2Vec_model_500.model.wv.most_similar("inshalla")

### 3. Model amb 1GB de dades

In [ ]:
Word2Vec_model_1024 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 1)

In [ ]:
Word2Vec_model.get_embedding("hola")

## Entrenament model de Similitud de Text Semàntic

### Imports

In [4]:
from importació_data import read_all_ts_data, reformat_data, create_corpus, preprocess, flattened_corpus_count

ImportError: cannot import name 'triu' from 'scipy.linalg' (C:\Users\Pol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\linalg\__init__.py)

- Llegim totes les dades de text similarity dividint-les en train, test i val. 
- Reformatejem les dades per a que siguin l'estructura List[Tuple[str, str, float]]. 
- Definim el corpus i el diccionari amb totes les paraules.
- Creem un diccionari de python amb tots els indexs com a claus i amb la repetició de les paraules com a valor.

In [ ]:
train, test, val = read_all_ts_data()
train, test, val = reformat_data(train, test, val)
corpus, all_words = create_corpus(train, test, val, preprocess=preprocess)
flat_corpus = flattened_corpus_count(corpus)

### Compartació amb diferents models de Word Embeddings

#### 1. One Hot

In [ ]:
from onehot import map_one_hot
from importació_data import pair_list_to_x_y
from model_bàsic import build_and_compile_model_better
import tensorflow as tf
from model_bàsic import compute_pearson

Un embedding OneHot té tamany igual a la llargada del diccionari. En la importació de dades ja hem eliminat les stopwords per reduïr la dimensió, però ara també eliminarem del embedding aquelles paraules que es repeteixen masses poques vegades o massa sovint. Per aconseguir això creem una llista que conté els indexs de les paraules que sí que utilitzarem i la passem com a argument a la funció *map_one_hot()*, per reduïr la dimensió del embedding.

In [ ]:
# Eliminate from all_words, the words that are repeated very few times or too much times
keys_preprocess = [index  for index in all_words if flat_corpus[index] > 10 and flat_corpus[index] < 50]

Convertim les paraules a vectors OneHot amb la funció *map_one_hot()*. Aquesta funció crea un vector de zeros de la mida del diccionari i posa un 1 a la posició de la paraula en el diccionari.

In [ ]:
mapped_one_hot_train = map_one_hot(train, all_words, keys_preprocess)
mapped_one_hot_test = map_one_hot(test, all_words, keys_preprocess)
mapped_one_hot_val = map_one_hot(val, all_words, keys_preprocess)

Separem el X i Y

In [ ]:
x_train, y_train = pair_list_to_x_y(mapped_one_hot_train)
x_val, y_val = pair_list_to_x_y(mapped_one_hot_val)

Entrenem el model

In [ ]:
batch_size: int = 64
num_epochs: int = 64

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
x_train[0].shape, x_train[1].shape, y_train.shape

In [ ]:
embedding_size = len(keys_preprocess)
model = build_and_compile_model_better(embedding_size = embedding_size)
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, )
print(model.summary())

In [ ]:
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

In [ ]:
print(f"Correlación de Pearson (train): {compute_pearson(model, x_train, y_train)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model, x_val, y_val)}")

Provem el model

In [ ]:
x_test, y_test = pair_list_to_x_y(mapped_one_hot_test)

In [ ]:
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (test): {compute_pearson(model, x_test, y_test)}")

#### 2. Word2Vec preentrenats 

#### 3. SpaCy

In [1]:
import spacy

In [5]:
!python -m spacy download ca_core_news_md


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     ---------------------------------------- 0.1/49.2 MB 3.2 MB/s eta 0:00:16
     ---------------------------------------- 0.2/49.2 MB 2.5 MB/s eta 0:00:20
     ---------------------------------------- 0.3/49.2 MB 2.5 MB/s eta 0:00:20
     ---------------------------------------- 0.6/49.2 MB 2.8 MB/s eta 0:00:18
      --------------------------------------- 0.8/49.2 MB 3.1 MB/s eta 0:00:16
      --------------------------------------- 1.1/49.2 MB 3.4 MB/s eta 0:00:15
     - -------------------------------------- 1.3/49.2 MB 3.5 MB/s eta 0:00:14
     - -------------------------------------- 1.6/49.2 MB 3.7 MB/s eta 0:00:13
     - -------------------------------------- 1.9/49.2 MB 3.9 MB/s eta 0:00:13
     - -------------------------------------- 2.2/49.2 MB 4.0 MB/s eta 0:00:12
     -- ------------------------------------- 2.6/49.2 MB 4.1 MB/s eta 0:00:12
     -- ------------------------------------- 3.1/49.2 MB 

In [7]:
nlp = spacy.load("ca_core_news_md")

array([ 0.68042  , -1.4599   ,  5.8433   ,  0.74058  ,  2.0949   ,
        0.18079  ,  1.8701   , -1.2873   ,  3.0511   , -0.28411  ,
        3.4147   ,  0.43923  , -2.8034   , -1.8566   , -1.2819   ,
       -2.8928   ,  0.6468   , -2.6281   ,  4.6685   , -0.99647  ,
       -1.1507   , -0.62254  ,  2.7648   , -3.0783   ,  1.3667   ,
        2.1095   ,  0.26997  , -0.081743 , -0.86469  ,  5.9948   ,
        2.4253   ,  1.5278   ,  1.5823   , -3.6996   , -3.8477   ,
       -0.14405  , -2.1246   ,  4.2239   , -1.6706   ,  0.43578  ,
       -0.6248   ,  1.5282   ,  4.207    , -0.051116 , -3.7832   ,
        0.0070785, -1.0093   ,  1.8353   , -1.2686   , -0.82724  ,
        2.6191   , -4.1626   ,  1.5366   , -0.49138  , -0.12545  ,
        0.51432  ,  1.3739   , -0.24285  , -1.1281   , -2.8424   ,
        0.82868  , -2.0375   , -0.76264  ,  1.9085   , -0.78051  ,
       -2.2556   , -1.5522   , -0.30314  , -0.73646  , -0.12398  ,
        0.14051  , -0.46373  , -1.6978   , -2.4986   ,  2.7556

#### 4. RoBERTa 

In [ ]:
nlp = spacy.load("ca_core_news_trf")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

#### 5. RoBERTa fine-tuned

In [ ]:
model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = [("El llibre va caure per la finestra.", "El llibre va sortir volant."),
                  ("M'agrades.", "T'estimo."),
                  ("M'agrada el sol i la calor", "A la Garrotxa plou molt.")]

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)